# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [2]:
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

llm = ChatOllama(model="qwen2:7b-instruct-q2_K", temperature=0)

In [3]:
from pathlib import Path

relevant_parts_pdf = []
for p in Path(".").absolute().parts:
    relevant_parts_pdf.append(p)
    if relevant_parts_pdf[-3:] == ["langchain", "docs", "modules"]:
        break
print(relevant_parts_pdf)
doc_path_pdf_01 = str(Path(*relevant_parts_pdf) / "CCI1010.pdf")
doc_path_pdf_02 = str(Path(*relevant_parts_pdf) / "CCI1477.pdf")
doc_path_pdf_03 = str(Path(*relevant_parts_pdf) / "CCI1.088_2024.pdf")
doc_path_pdf_01, doc_path_pdf_02, doc_path_pdf_03

['/', 'home', 'rogerio_rodrigues', 'python-workspace', 'rag_python', 'notes']


('/home/rogerio_rodrigues/python-workspace/rag_python/notes/CCI1010.pdf',
 '/home/rogerio_rodrigues/python-workspace/rag_python/notes/CCI1477.pdf',
 '/home/rogerio_rodrigues/python-workspace/rag_python/notes/CCI1.088_2024.pdf')

In [9]:
from typing import List
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader, PyPDFium2Loader
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_text_splitters.spacy import SpacyTextSplitter

text_splitter = SpacyTextSplitter(pipeline="pt_core_news_sm") # RecursiveCharacterTextSplitter(chunk_size=300)

def get_documents(file: str) -> List[Document]:
    __loader        = PyPDFium2Loader(file)
    __documents     = __loader.load()
    
    _new_docs = []
    for __doc in __documents:
        __text = __doc.page_content
        __text = __text.replace("#RESTRITA#", "")
        __doc.page_content = __text.replace('\n', '\n\n')
        _new_docs.append(__doc)
    return _new_docs

documents = []
documents.extend(get_documents(doc_path_pdf_01))
documents.extend(get_documents(doc_path_pdf_02))
documents.extend(get_documents(doc_path_pdf_03))
embeddings    = OllamaEmbeddings(model="nomic-embed-text")
store         = InMemoryStore()
docsearch     = Chroma.from_documents(documents, embeddings, collection_name="cci_pdf")

full_doc_retriever = ParentDocumentRetriever(
    vectorstore=docsearch,
    docstore=store,
    child_splitter=text_splitter
)

full_doc_retriever.add_documents(documents, ids=None)

comunicado_retrieve = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=full_doc_retriever, return_source_documents=True
)

comunicado_retrieve.verbose = True

/home/rogerio_rodrigues/.local/lib/python3.10/site-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [ ]:
docsearch.delete_collection()

In [10]:
for doc in documents:
    print('#---------------------')
    print(doc)
    print('#---------------------')

#---------------------
page_content='

CCI – 1.010/2024 - CCS Brasília/DF, 31 de julho de 2024.

Às entidades do Sicoob.

Evolução da funcionalidade Multicálculo Automóvel no Sistema Integrado para Gestão e 

Aquisição de Seguros do Sicoob (SicoobSigas) do Sisbr 3.0, referente às mudanças na 

integração das seguradoras Porto e Azul.

Senhores(as),

1. Com as mudanças na integração das seguradoras Porto e Azul, informamos a 

evolução da funcionalidade Multicálculo Automóvel no SicoobSigas do Sisbr 3.0, 

realizada em 24/7/2024, contemplando as seguintes alterações nas companhias:

a) a partir de 24/7/2024, os cálculos realizados não serão salvos. Após essa data, 

será necessário criar uma simulação para calcular ou contratar seguros;

b) nas renovações de seguros com vigência a partir de 23/8/2024, será 

apresentado um pacote fechado de condição exclusiva de cada seguradora. 

Não será válido o recálculo ou ajustes de coberturas;

c) para os casos em que há uma recusa no produto tra

In [12]:
comunicado_retrieve.invoke("""Faça um resumo do comunicado CCI - 1.477/2023""".strip())



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Faça um resumo do comunicado CCI - 1.477/2023',
 'result': 'O texto fornecido parece ser uma série de instruções ou declarações sobre como os envolvidos devem interagir com as cooperativas e entidades mencionadas nos documentos. Aqui estão os pontos principais:\n\n1. O número de telefone para contato é o "(61) 3771-6600", que pode ser usado para acessar o "Portal de Serviços do CCS".\n\n2. Os procedimentos e instruções podem ser encontrados no "Portal de Serviços do CCS".\n\n3. O código "IC xxxx" é referente a diferentes categorias, como acesso ao chat WhatsApp da entidade CCS ou parametrização/geração do benefício.\n\n4. Eventos ou pedidos de esclarecimentos devem ser encaminhados para o "(61) 3771-6600", opção específica e categorias específicas no portal.\n\n5. As cooperativas centrais podem estabelecer cronogramas regionais, levando em consideração os estágios do Plano de Ação até 2025 conforme a necessidade local.\n\n6. As PDEs (Pessoas Delegadas) são responsáveis pela 

In [ ]:
from langchain_community.document_loaders import TextLoader, PyPDFium2Loader

loader        = TextLoader(doc_path)
documents     = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts         = text_splitter.split_documents(documents)

embeddings    = OllamaEmbeddings(model="nomic-embed-text")
docsearch     = Chroma.from_documents(texts, embeddings, collection_name="cci")

In [ ]:
comunicado_retrieve = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [ ]:
docs       = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db    = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff       = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [ ]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent, ZeroShotAgent, create_react_agent

In [ ]:
tools = [
    Tool(
        name="Comunicado 1.010/2024 QA System",
        func=comunicado_retrieve.run,
        description="útil para quando você precisa responder a perguntas sobre o comunicado. A entrada deve ser uma pergunta totalmente formada.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.run(
    "A evolução da funcionalidade Multicálculo Automóvel se referente às mudanças na integração das seguradoras Porto e Azul?"
)

In [ ]:
agent.run("Why use ruff over flake8?")

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [ ]:
tools = [
    Tool(
        name="Comunicado 1.010/2024 QA System",
        func=comunicado_retrieve.run,
        description="útil para quando você precisa responder a perguntas sobre o comunicado. A entrada deve ser uma pergunta totalmente formada.",
        return_direct=True
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True
    ),
]

In [ ]:
agent = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.run(
    "A evolução da funcionalidade Multicálculo Automóvel se referente às mudanças na integração das seguradoras Porto e Azul?"
)

In [ ]:
agent.run("Why use ruff over flake8?")

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [ ]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.run(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)